In [ ]:
from SocialNetwork.social_network_environment import *
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
#from SocialNetwork.cascade import *
from Advertising.learners.Greedy_algorithm import *
from Advertising.learners.UCB_no_anything import *
from Advertising.learners.UCB_no_qualities import *

seed = 6789 
random.seed(seed)

exploration=4.2

deltas=[0.5,0.42,0.38,0.30,0.2,0.05]

social_network = social_network_environment()
social_network.init_from_json(json_path_network='Config/network.json', json_path_features='Config/features.json')

adversary_bids = []
for _ in range(0, 10): 
    adversary_i_bids = []
    for _ in range(0, 5):
        adversary_i_bids.append(random.randint(0, 4))
    adversary_bids.append(adversary_i_bids)

arms = arms_creation(seed = 30291, number_of_arms= 20)
n_rounds=10000
estimation_rounds=2000
all_bids = adversary_bids
vcg=VCG()
with open('Config/estimated_influence.json', 'r') as file:
    data = json.load(file)
    nodes_estimation = data.get('estimated_influence')
#nodes_estimation=compute_array_estimated_influence(social_network=social_network, rounds=estimation_rounds)

In [ ]:
#Clairvoyant for point 4, only first position
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for j in range(0, len(best_allocation)):
            temp_allocation = []
            temp_allocation = best_allocation[j]
            for k in range(0, len(best_allocation[j])):
                if temp_allocation[k].ad_id == 1 and k != 0:
                    temp_allocation[k].ad_id = None
                    best_allocation[j] = temp_allocation
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, reward_click = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
        reward_influence = 0
        seeds=np.where(active_nodes==1)[0]
        if len(seeds) != 0:
            for k in range(0, len(seeds)):
                reward_influence += nodes_estimation[seeds[k]]
            reward_influence = reward_influence/len(seeds)# + reward_influence)/2
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (reward_click+reward_influence-payments_tot) 
        reset_nodes(social_network)
    
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_q_f=max(all_exp_rewards)

In [ ]:
#Clairvoyant for point 4, with all positions
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, reward_click = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
        reward_influence = 0
        seeds=np.where(active_nodes==1)[0]
        if len(seeds) != 0:
            for k in range(0, len(seeds)):
                reward_influence += nodes_estimation[seeds[k]]
            reward_influence = reward_influence/len(seeds)# + reward_influence)/2
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (reward_click+reward_influence-payments_tot) 
        reset_nodes(social_network)
    
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_q_a=max(all_exp_rewards)

In [ ]:
#Clairvoyant for point 5, only first position
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for j in range(0, len(best_allocation)):
            temp_allocation = []
            temp_allocation = best_allocation[j]
            for k in range(0, len(best_allocation[j])):
                if temp_allocation[k].ad_id == 1 and k != 0:
                    temp_allocation[k].ad_id = None
                    best_allocation[j] = temp_allocation
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, total_reward = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (total_reward-payments_tot) 
        reset_nodes(social_network)
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_nq_f=max(all_exp_rewards)

In [ ]:
#Clairvoyant for point 5, with all positions
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, total_reward = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (total_reward-payments_tot) 
        reset_nodes(social_network)
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_nq_a=max(all_exp_rewards)
        

In [ ]:
from Advertising.learners.UCB_Learner import *

vcg = VCG()

ucb_nq_f = UCB_Learner(arms = arms, hyperpar=exploration)
pulls=0
for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))


    index=ucb_nq_f.pull_arm()
    best_arm = arms[index]


    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)

    for j in range(0, len(best_allocation)):
        temp_allocation = []
        temp_allocation = best_allocation[j]
        for k in range(0, len(best_allocation[j])):
            if temp_allocation[k].ad_id == 1 and k != 0:
                temp_allocation[k].ad_id = None
                best_allocation[j] = temp_allocation

    # Computing payments
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)

    total_reward, active_nodes = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)

    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)


    reward = (total_reward-payments_tot)
    

    ucb_nq_f.update(index, reward)

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, no qualities, first positions: "+str(arms[ucb_nq_f.return_best_arm()]))
print(ucb_nq_f.return_expected_values())


In [ ]:
from Advertising.learners.UCB_Learner import *

vcg = VCG()

ucb_nq_a = UCB_Learner(arms = arms, hyperpar=exploration)

for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))


    index=ucb_nq_a.pull_arm()
    best_arm = arms[index]


    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)

    # Computing payments
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)

    total_reward, active_nodes = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)

    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)


    reward = (total_reward-payments_tot)
    print(reward)


    ucb_nq_a.update(index, reward)

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, no qualities, all positions: "+str(arms[ucb_nq_a.return_best_arm()]))
print(ucb_nq_a.return_expected_values())

In [ ]:
from Advertising.learners.UCB_Learner_qualities import *

arms = arms_creation(seed = 30291, number_of_arms= 20)

vcg = VCG()

ucb_q_a = UCB_Learner_qualities(arms = arms, hyperpar=exploration, nodes_estimation=nodes_estimation)

for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))

    index = ucb_q_a.pull_arm()
    best_arm = arms[index]  

    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)  

    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
    active_nodes, click_rewards = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)

    reward = click_rewards - payments_tot

    ucb_q_a.update(index, reward, np.where(active_nodes==1)[0])

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, qualities, all positions: "+str(arms[ucb_q_a.return_best_arm()]))

In [ ]:
from Advertising.learners.UCB_Learner_qualities import *

arms = arms_creation(seed = 30291, number_of_arms= 20)

vcg = VCG()

ucb_q_f = UCB_Learner_qualities(arms = arms, hyperpar=exploration, nodes_estimation = nodes_estimation)

for t in range(0, n_rounds):
    if t%500 == 0:
        print("Still Going at round " + str(t))

    index = ucb_q_f.pull_arm()
    best_arm = arms[index]  

    all_bids.insert(0, best_arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)  

    for j in range(0, len(best_allocation)):
        temp_allocation = []
        temp_allocation = best_allocation[j]
        for k in range(0, len(best_allocation[j])):
            if temp_allocation[k].ad_id == 1 and k != 0:
                temp_allocation[k].ad_id = None
                best_allocation[j] = temp_allocation
    
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
    active_nodes, click_rewards = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)

    reward = click_rewards - payments_tot

    ucb_q_f.update(index, reward, np.where(active_nodes==1)[0])

    all_bids.pop(0)
    reset_nodes(social_network)

print("Optimal arm, qualities, first positions: "+str(arms[ucb_q_f.return_best_arm()]))